In [1]:
import os
import sys
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import portion as P
import itertools as it
import csv
from pprint import pprint
from pytictoc import TicToc
from tqdm import tqdm
import traceback
import math

# ******************************* User Settings *******************************
database = "/home/wmnlab/D/database/"
date = "2022-11-29"
devices = sorted([
    # "sm00",
    # "sm01",
    # "sm02",
    # "sm03",
    # "sm04",
    "sm05",
    "sm06",
    "sm07",
    "sm08",
    # "qc00",
    # "qc01",
    # "qc02",
    # "qc03",
])
exps = {  # experiment_name: (number_of_experiment_rounds, list_of_experiment_round)
            # If the list is None, it will not list as directories.
            # If the list is empty, it will list all directories in the current directory by default.
            # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    # "tsync": (1, None),
    # "_Bandlock_Udp": (4, ["#01", "#02", "#03", "#04"]),
    # "_Bandlock_Udp": (4, ["#03", "#04", "#05", "#06"]),
    # "_Bandlock_Udp": (4, []),
    # "_Bandlock_Udp": (6, []),
    "_Bandlock_Udp_B1_B3":  (4, []),
    "_Bandlock_Udp_B3_B28": (4, []),
    "_Bandlock_Udp_B28_B1": (4, []),
}

class Payload:
    LENGTH = 250              # (Bytes)
    TAG = "000425d401df5e76"  # 2 71828 3 1415926 (hex)            : 8-bytes
    OFS_TIME = (16, 24)       # epoch time of 'yyyy/mm/dd hh:mm:ss': 4-bytes
    OFS_USEC = (24, 32)       # microsecond (usec)                 : 4-bytes
    OFS_SEQN = (32, 40)       # sequence number (start from 1)     : 4-bytes
class ServerIP:
    PUBLIC = "140.112.20.183"  # 2F    
    PRIVATE = "192.168.1.251"  # 2F
    # PRIVATE = "192.168.1.248"  # 2F previous
    # PUBLIC = "140.112.17.209"  # 3F
    # PRIVATE = "192.168.1.108"  # 3F

DATA_RATE = 1000e3  # bits-per-second
PKT_RATE = DATA_RATE / Payload.LENGTH / 8  # packets-per-second
print("packet_rate (pps):", PKT_RATE, "\n")
# *****************************************************************************

packet_rate (pps): 500.0 



In [2]:
# ****************************** Utils Functions ******************************
def makedir(dirpath, mode=0):  # mode=1: show message; mode=0: hide message
    if os.path.isdir(dirpath):
        if mode:
            print("mkdir: cannot create directory '{}': directory has already existed.".format(dirpath))
        return
    ### recursively make directory
    _temp = []
    while not os.path.isdir(dirpath):
        _temp.append(dirpath)
        dirpath = os.path.dirname(dirpath)
    while _temp:
        dirpath = _temp.pop()
        print("mkdir", dirpath)
        os.mkdir(dirpath)
# *****************************************************************************

In [3]:
handover_types = "lte_handover,SN_addition,SN_removal,endc_SN_change,endc_MN_change,endc_MNSN_change,lte2endc_MN_change,endc2lte_MN_change".split(',')
link_failure_types = "scg_failure,radio_link_failure,nas_recovery".split(',')
handover_types_1 = "SN_change_only,Intra_frequency,Inter_frequency,-".split(',')
handover_types_2 = "SN_change_only,Intra_sector,Intra_eNB,Inter_eNB,-".split(',')
print(handover_types)
print(link_failure_types)
print(handover_types_1)
print(handover_types_2)

['lte_handover', 'SN_addition', 'SN_removal', 'endc_SN_change', 'endc_MN_change', 'endc_MNSN_change', 'lte2endc_MN_change', 'endc2lte_MN_change']
['scg_failure', 'radio_link_failure', 'nas_recovery']
['SN_change_only', 'Intra_frequency', 'Inter_frequency', '-']
['SN_change_only', 'Intra_sector', 'Intra_eNB', 'Inter_eNB', '-']


In [4]:
# **************************** Auxiliary Functions ****************************
def interp(x, y, ratio):
    """
    Interpolation

    Args:
        x, y (datetime.datetime)
        ratio (float): a decimal numeral in a range [0, 1]
    Returns:
        (datetime.datetime): breakpoint of interpolation
    """
    return x + (y - x) * ratio

def is_disjoint(set1, set2):
    """
    Check if two sets are disjoint.
    """
    return (set1 & set2).empty

def all_disjoint(set_list):
    """
    Check if all sets in the list are pairwise disjoint.
    """
    pair_list = list(it.combinations(set_list, 2))
    for item in pair_list:
        if not is_disjoint(item[0], item[1]):
            return False
    return True

def all_union(set_list):
    """
    Union all set in the list.
    """
    _temp = P.empty()
    for _set in set_list:
        _temp = _temp | _set
    return _temp

def get_handover_intervals(df, types0=handover_types, secs=(0, 1), types1=handover_types_1, types2=handover_types_2, exp_time=(dt.datetime.min, dt.datetime.max), overlap=True, ratio=1):
    """
    Get intervals of each event type.

    Args:
        df (pandas.Dataframe): dataframe read from diag_log_ho-info.csv
        event_names (list): list of event types, it would include all spcified types together.
        secs (float): size of window that you need. (in seconds)
        exp_time (tuple): (start_time, end_time) of an experiment.
        overlap (bool): allows intervals to be overlapping or not.
        ratio (float): a decimal numeral in a range [0, 1], to decide the breakpoint of interpolation.

    Example. secs=1.2, ratio=0.8
        .                    (overlap)               (overlap)
    ----[------------)[.](--[----------]--)[x](-------[-----]-------)[.](------------]----> (sec)
        1.b            1    2.b      a.1    2         3.b a.2         3            a.3
        .
    ----[------------)[.](----------)[----)[x](-----------)[--------)[.](------------]----> (sec)
        1.b            1          a.12.b    2           a.23.b        3            a.3
        We assume the during events of each type do not overlap, the previous or next handover state would only be the following,
        .] : 'end' or 'trigger'
        [x]: 'start'-'end' pair or 'trigger
        [. : 'start' or 'trigger
        'a' means "after_event", 'b' means "before_event"
    
    Extreme Case (1). secs=1.2, ratio=0 (before/prior case) 後者全拿
        .
    ----[--------[----)   [.]   (--------)   [x]   (----]--------]----> (sec)
        1.b      2.b       1                  2       a.1      a.2
        .        |(bp)          |(.])
    ----[-------- ----)   [.] ()(--------)   [x]   (---- --------]----> (sec)
        1.b                1 a.12.b           2                a.2
        take max{ bp & .] } as left-bound, a.1 would be empty()
        'a' means "after_event", 'b' means "before_event"
    
    Extreme Case (2). secs=1.2, ratio=1 (after/post case) 前者全拿
    .
    ----[--------[----)   [x]   (--------)   [.]   (----]--------]----> (sec)
        1.b      2.b       1                  2       a.1      a.2
        .                                |([.)          |(bp)
    ----[-------- ----)   [x]   (--------)() [.]   (---- --------]----> (sec)
        1.b                1           a.12.b 2                a.2
        take min{ bp & [. } as right-bound, 2.b would be empty()
        'a' means "after_event", 'b' means "before_event"

    Returns:
        before_event_intervals (portion.interval.Interval)
        during_events_intervals (portion.interval.Interval)
        after_events_intervals (portion.interval.Interval)
    """
    before_events_intervals = P.empty()
    during_events_intervals = P.empty()
    after_events_intervals = P.empty()
    _prior_interval = P.empty()
    _midd_interval = P.empty()
    _post_interval = P.empty()
    # print(df)
    anomaly_check = []
    for i in range(len(df)):
        # print(i+2)
        # if df.loc[i, 'handoff_type'] in event_names:
        if df.loc[i, 'handoff_type'] in types0 and df.loc[i, 'handoff_type.1'] in types1 and df.loc[i, 'handoff_type.2'] in types2:
            if df.loc[i, 'handoff_state'] == 'start':
                ### before
                _start = df.iloc[i]
                # prior_interval = P.closedopen( max(_start['Timestamp'] - dt.timedelta(seconds=secs), exp_time[0]), min(_start['Timestamp'], exp_time[1]) )
                prior_interval = P.closedopen( max(_start['Timestamp'] - dt.timedelta(seconds=secs[1]), exp_time[0]), min(_start['Timestamp'] - dt.timedelta(seconds=secs[0]), exp_time[1]) )
                if not overlap and i != 0 and (df.loc[i-1, 'Timestamp'] > prior_interval.lower):
                # if not overlap and i != 0 and (df.loc[i-1, 'Timestamp'] + dt.timedelta(seconds=secs) in prior_interval or df.loc[i-1, 'Timestamp'] in prior_interval):
                # if not overlap and i not in [0, len(df)-1] and (df.loc[i-1, 'Timestamp'] + dt.timedelta(seconds=secs) in prior_interval or df.loc[i-1, 'Timestamp'] in prior_interval):
                    # print(i+2, "start prior overlap")
                    # breakpoint = interp(_start['Timestamp'] - dt.timedelta(seconds=secs), df.loc[i-1, 'Timestamp'] + dt.timedelta(seconds=secs), ratio)
                    breakpoint = interp(_start['Timestamp'] - dt.timedelta(seconds=secs[1]), df.loc[i-1, 'Timestamp'], ratio=1)
                    breakpoint = max(breakpoint, df.loc[i-1, 'Timestamp'])
                    ### All open is fine, but sometimes may miss one point.
                    # prior_interval = P.closedopen( max(breakpoint, exp_time[0]), min(_start['Timestamp'], exp_time[1]) )
                    prior_interval = P.closedopen( max(breakpoint, exp_time[0]), min(_start['Timestamp'] - dt.timedelta(seconds=secs[0]), exp_time[1]) )
                    if df.loc[i-1, 'Timestamp'] in prior_interval:
                        # prior_interval = P.open( max(breakpoint, exp_time[0]), min(_start['Timestamp'], exp_time[1]) )
                        prior_interval = P.open( max(breakpoint, exp_time[0]), min(_start['Timestamp'] - dt.timedelta(seconds=secs[0]), exp_time[1]) )
                before_events_intervals = before_events_intervals | prior_interval
                ### Exception: during, after
                if i == len(df)-1:
                    midd_interval = P.closed( max(_start['Timestamp'], exp_time[0]), exp_time[1] )
                    during_events_intervals = during_events_intervals | midd_interval
                    post_interval = P.empty()
                    after_events_intervals = after_events_intervals | post_interval
            elif df.loc[i, 'handoff_state'] == 'end':
                ### Exception: before
                if i == 0:
                    prior_interval = P.empty()
                    before_events_intervals = before_events_intervals | prior_interval
                    _start = pd.Series({"Timestamp":exp_time[0], "type_id":'-', "handoff_type":'-', "handoff_state":'-', "handoff_duration":'-', "PCI":'-', "EARFCN":'-', "NR_PCI":'-', "NR_ARFCN":'-'})
                ### during
                midd_interval = P.closed( max(_start['Timestamp'], exp_time[0]), min(df.loc[i, 'Timestamp'], exp_time[1]) )
                during_events_intervals = during_events_intervals | midd_interval
                ### after
                # post_interval = P.openclosed( max(df.loc[i, 'Timestamp'], exp_time[0]), min(df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs), exp_time[1]) )
                post_interval = P.openclosed( max(df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs[0]), exp_time[0]), min(df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs[1]), exp_time[1]) )
                if not overlap and i != len(df)-1 and (df.loc[i+1, 'Timestamp'] < post_interval.upper):
                # if not overlap and i != len(df)-1 and (df.loc[i+1, 'Timestamp'] - dt.timedelta(seconds=secs) in post_interval or df.loc[i+1, 'Timestamp'] in post_interval):
                # if not overlap and i not in [0, len(df)-1] and (df.loc[i+1, 'Timestamp'] - dt.timedelta(seconds=secs) in post_interval or df.loc[i+1, 'Timestamp'] in post_interval):
                    # print(i+2, "end post overlap")
                    # breakpoint = interp(df.loc[i+1, 'Timestamp'] - dt.timedelta(seconds=secs), df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs), ratio)
                    breakpoint = interp(df.loc[i+1, 'Timestamp'], df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs[1]), ratio=0)
                    breakpoint = min(breakpoint, df.loc[i+1, 'Timestamp'])
                    # post_interval = P.open( max(df.loc[i, 'Timestamp'], exp_time[0]), min(breakpoint, exp_time[1]) )
                    post_interval = P.open( max(df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs[0]), exp_time[0]), min(breakpoint, exp_time[1]) )
                after_events_intervals = after_events_intervals | post_interval
            elif df.loc[i, 'handoff_state'] == 'trigger':
                if i != 0 and df.loc[i-1, 'handoff_state'] == 'start':
                    # prior_interval = P.empty()
                    # post_interval = P.empty()
                    print("*******************************************************************")
                    print(df.loc[i, 'Timestamp'])
                    print("Link failure occurs during handover duration: {}".format(df.loc[i, 'handoff_type']))
                    print("Prior event: {} {}".format(df.loc[i-1, 'handoff_type'], df.loc[i-1, 'handoff_state']))
                    anomaly_check.append((df.loc[i, 'Timestamp'], df.loc[i, 'handoff_type'], df.loc[i-1, 'handoff_type']))
                    continue
                ### before
                # prior_interval = P.closedopen( max(df.loc[i, 'Timestamp'] - dt.timedelta(seconds=secs), exp_time[0]), min(df.loc[i, 'Timestamp'], exp_time[1]) )
                prior_interval = P.closedopen( max(df.loc[i, 'Timestamp'] - dt.timedelta(seconds=secs[1]), exp_time[0]), min(df.loc[i, 'Timestamp'] - dt.timedelta(seconds=secs[0]), exp_time[1]) )
                if not overlap and i != 0 and (df.loc[i-1, 'Timestamp'] > prior_interval.lower):
                # if not overlap and i != 0 and (df.loc[i-1, 'Timestamp'] + dt.timedelta(seconds=secs) in prior_interval or df.loc[i-1, 'Timestamp'] in prior_interval):
                # if not overlap and i not in [0, len(df)-1] and (df.loc[i-1, 'Timestamp'] + dt.timedelta(seconds=secs) in prior_interval or df.loc[i-1, 'Timestamp'] in prior_interval):
                    # print(i+2, "trigger prior overlap")
                    # breakpoint = interp(df.loc[i, 'Timestamp'] - dt.timedelta(seconds=secs), df.loc[i-1, 'Timestamp'] + dt.timedelta(seconds=secs), ratio)
                    breakpoint = interp(df.loc[i, 'Timestamp'] - dt.timedelta(seconds=secs[1]), df.loc[i-1, 'Timestamp'], ratio=1)
                    breakpoint = max(breakpoint, df.loc[i-1, 'Timestamp'])
                    ### All open is fine, but sometimes may miss one point.
                    # prior_interval = P.closedopen( max(breakpoint, exp_time[0]), min(df.loc[i, 'Timestamp'], exp_time[1]) )
                    prior_interval = P.closedopen( max(breakpoint, exp_time[0]), min(df.loc[i, 'Timestamp'] - dt.timedelta(seconds=secs[0]), exp_time[1]) )
                    if df.loc[i-1, 'Timestamp'] in prior_interval:
                        # prior_interval = P.open( max(breakpoint, exp_time[0]), min(df.loc[i, 'Timestamp'], exp_time[1]) )
                        prior_interval = P.open( max(breakpoint, exp_time[0]), min(df.loc[i, 'Timestamp'] - dt.timedelta(seconds=secs[0]), exp_time[1]) )
                before_events_intervals = before_events_intervals | prior_interval
                ### during
                midd_interval = P.singleton( df.loc[i, 'Timestamp'] )
                during_events_intervals = during_events_intervals | midd_interval
                ### after
                # post_interval = P.openclosed( max(df.loc[i, 'Timestamp'], exp_time[0]), min(df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs), exp_time[1]) )
                post_interval = P.openclosed( max(df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs[0]), exp_time[0]), min(df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs[1]), exp_time[1]) )
                # print('******************************************************')
                # print(df.loc[i, 'handoff_type'], df.loc[i, 'handoff_state'], df.loc[i, 'Timestamp'])
                # print(df.loc[i+1, 'handoff_type'], df.loc[i+1, 'handoff_state'], df.loc[i+1, 'Timestamp'])
                # print('******************************************************')
                if not overlap and i != len(df)-1 and (df.loc[i+1, 'Timestamp'] < post_interval.upper):
                # if not overlap and i != len(df)-1 and (df.loc[i+1, 'Timestamp'] - dt.timedelta(seconds=secs) in post_interval or df.loc[i+1, 'Timestamp'] in post_interval):
                # if not overlap and i not in [0, len(df)-1] and (df.loc[i+1, 'Timestamp'] - dt.timedelta(seconds=secs) in post_interval or df.loc[i+1, 'Timestamp'] in post_interval):
                    # print(i+2, "trigger post overlap")
                    # breakpoint = interp(df.loc[i+1, 'Timestamp'] - dt.timedelta(seconds=secs), df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs), ratio)
                    breakpoint = interp(df.loc[i+1, 'Timestamp'], df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs[1]), ratio=0)
                    breakpoint = min(breakpoint, df.loc[i+1, 'Timestamp'])
                    post_interval = P.open( max(df.loc[i, 'Timestamp'] + dt.timedelta(seconds=secs[0]), exp_time[0]), min(breakpoint, exp_time[1]) )
                after_events_intervals = after_events_intervals | post_interval

            # if df.loc[i, 'handoff_state'] == 'end' or df.loc[i, 'handoff_state'] == 'trigger':
            #     set1 = _prior_interval | _midd_interval | _post_interval
            #     set2 = prior_interval | midd_interval | post_interval
            #     # if not is_disjoint(set1, set2) or i+2 == 7:
            #     if not is_disjoint(set1, set2):
            #         print(False, i+2)
            #         print("-------------------------------------")
            #         print("previous:      ", set1)
            #         print("previous prior:", _prior_interval, is_disjoint(_prior_interval, set2))
            #         print("previous midd: ", _midd_interval, is_disjoint(_midd_interval, set2))
            #         print("previous post: ", _post_interval, is_disjoint(_post_interval, set2))
            #         print("-------------------------------------")
            #         print("current:       ", set2)
            #         print("current prior: ", prior_interval, is_disjoint(prior_interval, set1))
            #         print("current midd:  ", midd_interval, is_disjoint(midd_interval, set1))
            #         print("current post:  ", post_interval, is_disjoint(post_interval, set1))
            #         print("-------------------------------------")
            #     _prior_interval = prior_interval
            #     _midd_interval = midd_interval
            #     _post_interval = post_interval

    return before_events_intervals, during_events_intervals, after_events_intervals, anomaly_check

def get_intervals_length(intervals):
    """
    Get total length of a set of intervals.
    """
    if intervals.empty:
        return 0
    sum = 0
    for s in intervals:
        sum += (s.upper - s.lower) / dt.timedelta(seconds=1)
    return round(sum, 6)
# *****************************************************************************

In [5]:
def get_sst(df_dlloss, df_dlrecv, df_ulloss, df_ulrecv):
    start_list = []
    stop_list = []
    for df in [df_dlloss, df_dlrecv, df_ulloss, df_ulrecv]:
        if len(df):
            start_list.append(df["Timestamp"].iloc[0])
            stop_list.append(df["Timestamp"].iloc[-1])
    start = min(start_list) if len(start_list) else dt.datetime.min
    stop = min(stop_list) if len(stop_list) else dt.datetime.max
    expr_time = (stop - start).total_seconds() if len(start_list) and len(stop_list) else 0
    return start, stop, expr_time

def remove_head_tail(hodf, start_time, stop_time):
    if len(hodf):
        start_indices = hodf.index[hodf['Timestamp'] >= start_time]
        end_indices = hodf.index[hodf['Timestamp'] <= stop_time]
        if len(start_indices) and len(end_indices):
            start_index = start_indices[0]
            end_index = end_indices[-1]
        else:
            hodf = hodf.iloc[0:0]
        try:
            if hodf.loc[start_index, 'handoff_state'] == 'end':
                hodf.loc[start_index - 1, 'Timestamp'] = start_time
                start_index -= 1
            if hodf.loc[end_index, 'handoff_state'] == 'start':
                hodf.loc[end_index + 1, 'Timestamp'] = stop_time
                end_index += 1
            hodf = hodf.iloc[start_index : end_index + 1]
        except:
            pass
        hodf = hodf.reset_index(drop=True)
    return hodf

def handoff_statistics(hodf, expr_time, fout1, fout2):
    hodf.to_csv(fout2, index=False)
    hodf = hodf[hodf['handoff_state'] != 'end']
    event_names = "lte_handover,SN_addition,SN_removal,endc_SN_change,endc_MN_change,endc_MNSN_change,lte2endc_MN_change,endc2lte_MN_change,scg_failure,radio_link_failure,nas_recovery".split(',')
    ss = [0] * len(event_names)
    for i, item in enumerate(event_names):
        ss[i] = sum(hodf['handoff_type'] == item)
    ss.append(sum(ss[:8]))
    ss.append(sum(ss[8:11]))
    ss.append(sum(ss[:11]))
    ss.append(expr_time)
    event_names += ["succ_handoff", "fail_handoff", "overall_handoff", "experiment_time(sec)"]
    with open(fout1, "w", newline='') as fp:
        writer = csv.writer(fp)
        writer.writerow(event_names)
        writer.writerow(ss)

def loss_classify_type0(hodf, lossdf, recvdf, fout, start_time, stop_time, overlap=True, ratio=1):
    # anomaly_check_list = []
    handover_type_names = "lte_handover, SN_addition, SN_removal, endc_SN_change, endc_MN_change, endc_MNSN_change, lte2endc_MN_change, endc2lte_MN_change".split(', ')
    failure_type_names = "scg_failure, radio_link_failure, nas_recovery".split(', ')

    column_names = []
    for type_name in handover_type_names:
        column_names += ["before_{}".format(type_name), "during_{}".format(type_name), "after_{}".format(type_name)]
    for type_name in failure_type_names:
        column_names += ["before_{}".format(type_name), "after_{}".format(type_name)]
    column_names += ["before_succ_handoff", "during_succ_handoff", "after_succ_handoff", "before_fail_handoff", "after_fail_handoff", "unstable", "stable", "overall"]

    datarows = []
    ss0 = [0/10] + [i/10 for i in range(0, 10)] + list(range(1, 10))
    ss1 = [i/10 for i in range(0, 10)] + list(range(1, 11))
    for secs0, secs1 in zip(ss0, ss1):
        event_occurrence = [0] * len(column_names)
        sum_total_time = [0] * len(column_names)
        sum_total_num = [0] * len(column_names)
        sum_loss_num = [0] * len(column_names)
        loss_rate = [0] * len(column_names)

        all_intervals = []

        offset = len(handover_type_names) * 3
        for i, event_type in enumerate(handover_type_names):
            before_intervals, during_intervals, after_intervals, anomaly_check = get_handover_intervals(hodf, types0=[event_type], secs=(secs0, secs1), overlap=overlap, ratio=ratio)
            all_intervals.append(before_intervals)
            all_intervals.append(during_intervals)
            all_intervals.append(after_intervals)
            event_occurrence[3*i] = event_occurrence[3*i+1] = event_occurrence[3*i+2] = len(hodf[(hodf['handoff_type'] == event_type) & (hodf['handoff_state'] != 'end')])
            # if anomaly_check:
                # anomaly_check_list.append(anomaly_check)
        for i, event_type in enumerate(failure_type_names):
            before_intervals, during_intervals, after_intervals, anomaly_check = get_handover_intervals(hodf, types0=[event_type], secs=(secs0, secs1), overlap=overlap, ratio=ratio)
            all_intervals.append(before_intervals)
            all_intervals.append(after_intervals)
            event_occurrence[2*i+offset] = event_occurrence[2*i+offset+1] = len(hodf[(hodf['handoff_type'] == event_type) & (hodf['handoff_state'] != 'end')])
            # if anomaly_check:
                # anomaly_check_list.append(anomaly_check)
        print("disjoint set?", "yes!" if all_disjoint(all_intervals) else "no!")
        print("-----------------------------")

        all_intervals.append(all_union(all_intervals[:24:3]))
        all_intervals.append(all_union(all_intervals[1:24:3]))
        all_intervals.append(all_union(all_intervals[2:24:3]))
        all_intervals.append(all_union(all_intervals[24:30:2]))
        all_intervals.append(all_union(all_intervals[25:30:2]))
        all_intervals.append(all_union(all_intervals[:30]))
        all_intervals.append(P.closed(start_time, stop_time) - all_intervals[-1] if len(recvdf) else P.empty())
        all_intervals.append(P.closed(start_time, stop_time) if len(recvdf) else P.empty())

        loss_timestamps = lossdf['Timestamp'].array
        recv_timestamps = recvdf['Timestamp'].array
        for i, item in enumerate(all_intervals):
            sum_total_time[i] = get_intervals_length(item)
            # sum_total_num[i] = round(sum_total_time[i] * PKT_RATE)
            sum_total_num[i] = math.ceil(sum_total_time[i] * PKT_RATE)
            for j in range(len(lossdf)):
                if item.empty:
                    break
                # if lossdf['Timestamp'].iloc[j] > item.upper:
                if loss_timestamps[j] > item.upper:
                    break
                # if lossdf['Timestamp'].iloc[j] in item:
                if loss_timestamps[j] in item:
                    sum_loss_num[i] += 1
            ### 絕無誤差的計算 sent_packet 數量的方式（取消註解即可）
            # sum_total_num[i] = 0
            # for j in range(len(recvdf)):
            #     if item.empty:
            #         break
            #     # if recvdf['Timestamp'].iloc[j] > item.upper:
            #     if recv_timestamps[j] > item.upper:
            #         break
            #     # if recvdf['Timestamp'].iloc[j] in item:
            #     if recv_timestamps[j] in item:
            #         sum_total_num[i] += 1
            # sum_total_num[i] += sum_loss_num[i]
            ######
            loss_rate[i] = sum_loss_num[i] / (sum_total_num[i] + 1e-9) * 100
        
        event_occurrence[-6] = event_occurrence[-7] = event_occurrence[-8] = sum(event_occurrence[:24:3])  # successful handoff
        event_occurrence[-4] = event_occurrence[-5] = sum(event_occurrence[24:30:2])  # failed handoff
        event_occurrence[-3] = event_occurrence[-5] + event_occurrence[-8]  # all handoff (unstable)
        event_occurrence[-1] = event_occurrence[-2] = '-'  # stable & overall

        # datarow = [str(secs)]
        # for i in range(len(column_names)):
        #     datarow.append('@'.join([str(event_occurrence[i]), str(sum_total_time[i]), str(sum_loss_num[i]), str(sum_total_num[i]), str(loss_rate[i])]))
        # datarows.append(datarow)

        datarow = ["{}-{}".format(secs0, secs1), "#event"] + event_occurrence
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "total_duration"] + sum_total_time
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "#packet_loss"] + sum_loss_num
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "#total_packets"] + sum_total_num
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "packet_loss_rate"] + loss_rate
        datarows.append(datarow)

    print("output >>>", fout)
    with open(fout, "w", newline='') as fp:
        writer = csv.writer(fp)
        # writer.writerow(["secs"] + column_names)
        writer.writerow(["secs", "metrics"] + column_names)
        writer.writerows(datarows)

def loss_classify_type1(hodf, lossdf, recvdf, fout, start_time, stop_time, overlap=True, ratio=1):
    # anomaly_check_list = []
    handover_type_names = "SN_change_only,Intra_frequency,Inter_frequency".split(',')
    # handover_types_1 = "SN_change_only,Intra_frequency,Inter_frequency,-".split(',')
    # handover_types_2 = "SN_change_only,Intra_sector,Intra_eNB,Inter_eNB,-".split(',')
    
    column_names = []
    for type_name in handover_type_names:
        column_names += ["before_{}".format(type_name), "during_{}".format(type_name), "after_{}".format(type_name)]
    column_names += ["before_handoff", "during_handoff", "after_handoff", "unstable", "stable", "overall"]

    datarows = []
    ss0 = [0/10] + [i/10 for i in range(0, 10)] + list(range(1, 10))
    ss1 = [i/10 for i in range(0, 10)] + list(range(1, 11))
    for secs0, secs1 in zip(ss0, ss1):
        event_occurrence = [0] * len(column_names)
        sum_total_time = [0] * len(column_names)
        sum_total_num = [0] * len(column_names)
        sum_loss_num = [0] * len(column_names)
        loss_rate = [0] * len(column_names)

        all_intervals = []

        offset = len(handover_type_names) * 3
        # print(hodf)
        for i, event_type in enumerate(handover_type_names):
            # print(event_type)
            before_intervals, during_intervals, after_intervals, anomaly_check = get_handover_intervals(hodf, types1=[event_type], secs=(secs0, secs1), overlap=overlap, ratio=ratio)
            all_intervals.append(before_intervals)
            all_intervals.append(during_intervals)
            all_intervals.append(after_intervals)
            event_occurrence[3*i] = event_occurrence[3*i+1] = event_occurrence[3*i+2] = len(hodf[(hodf['handoff_type.1'] == event_type) & (hodf['handoff_state'] != 'end')])
            # if anomaly_check:
                # anomaly_check_list.append(anomaly_check)
        print("disjoint set?", "yes!" if all_disjoint(all_intervals) else "no!")
        print("-----------------------------")

        all_intervals.append(all_union(all_intervals[:9:3]))
        all_intervals.append(all_union(all_intervals[1:9:3]))
        all_intervals.append(all_union(all_intervals[2:9:3]))
        all_intervals.append(all_union(all_intervals[:9]))
        all_intervals.append(P.closed(start_time, stop_time) - all_intervals[-1] if len(recvdf) else P.empty())
        all_intervals.append(P.closed(start_time, stop_time) if len(recvdf) else P.empty())

        loss_timestamps = lossdf['Timestamp'].array
        recv_timestamps = recvdf['Timestamp'].array
        for i, item in enumerate(all_intervals):
            sum_total_time[i] = get_intervals_length(item)
            # sum_total_num[i] = round(sum_total_time[i] * PKT_RATE)
            sum_total_num[i] = math.ceil(sum_total_time[i] * PKT_RATE)
            for j in range(len(lossdf)):
                if item.empty:
                    break
                # if lossdf['Timestamp'].iloc[j] > item.upper:
                if loss_timestamps[j] > item.upper:
                    break
                # if lossdf['Timestamp'].iloc[j] in item:
                if loss_timestamps[j] in item:
                    sum_loss_num[i] += 1
            ### 絕無誤差的計算 sent_packet 數量的方式（取消註解即可）
            # sum_total_num[i] = 0
            # for j in range(len(recvdf)):
            #     if item.empty:
            #         break
            #     # if recvdf['Timestamp'].iloc[j] > item.upper:
            #     if recv_timestamps[j] > item.upper:
            #         break
            #     # if recvdf['Timestamp'].iloc[j] in item:
            #     if recv_timestamps[j] in item:
            #         sum_total_num[i] += 1
            # sum_total_num[i] += sum_loss_num[i]
            ######
            loss_rate[i] = sum_loss_num[i] / (sum_total_num[i] + 1e-9) * 100
        
        event_occurrence[-4] = event_occurrence[-5] = event_occurrence[-6] = sum(event_occurrence[:9:3])  # successful handoff
        # event_occurrence[-4] = event_occurrence[-5] = sum(event_occurrence[24:30:2])  # failed handoff
        # event_occurrence[-3] = event_occurrence[-5] + event_occurrence[-8]  # all handoff (unstable)
        event_occurrence[-3] = event_occurrence[-6]  # all handoff (unstable)
        event_occurrence[-1] = event_occurrence[-2] = '-'  # stable & overall

        # datarow = [str(secs)]
        # for i in range(len(column_names)):
        #     datarow.append('@'.join([str(event_occurrence[i]), str(sum_total_time[i]), str(sum_loss_num[i]), str(sum_total_num[i]), str(loss_rate[i])]))
        # datarows.append(datarow)

        datarow = ["{}-{}".format(secs0, secs1), "#event"] + event_occurrence
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "total_duration"] + sum_total_time
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "#packet_loss"] + sum_loss_num
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "#total_packets"] + sum_total_num
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "packet_loss_rate"] + loss_rate
        datarows.append(datarow)

    print("output >>>", fout)
    with open(fout, "w", newline='') as fp:
        writer = csv.writer(fp)
        # writer.writerow(["secs"] + column_names)
        writer.writerow(["secs", "metrics"] + column_names)
        writer.writerows(datarows)

def loss_classify_type2(hodf, lossdf, recvdf, fout, start_time, stop_time, overlap=True, ratio=1):
    # anomaly_check_list = []
    handover_type_names = "SN_change_only,Intra_sector,Intra_eNB,Inter_eNB".split(',')
    # handover_types_1 = "SN_change_only,Intra_frequency,Inter_frequency,-".split(',')
    # handover_types_2 = "SN_change_only,Intra_sector,Intra_eNB,Inter_eNB,-".split(',')
    
    column_names = []
    for type_name in handover_type_names:
        column_names += ["before_{}".format(type_name), "during_{}".format(type_name), "after_{}".format(type_name)]
    column_names += ["before_handoff", "during_handoff", "after_handoff", "unstable", "stable", "overall"]

    datarows = []
    ss0 = [0/10] + [i/10 for i in range(0, 10)] + list(range(1, 10))
    ss1 = [i/10 for i in range(0, 10)] + list(range(1, 11))
    for secs0, secs1 in zip(ss0, ss1):
        event_occurrence = [0] * len(column_names)
        sum_total_time = [0] * len(column_names)
        sum_total_num = [0] * len(column_names)
        sum_loss_num = [0] * len(column_names)
        loss_rate = [0] * len(column_names)

        all_intervals = []

        offset = len(handover_type_names) * 3
        # print(hodf)
        for i, event_type in enumerate(handover_type_names):
            # print(event_type)
            before_intervals, during_intervals, after_intervals, anomaly_check = get_handover_intervals(hodf, types2=[event_type], secs=(secs0, secs1), overlap=overlap, ratio=ratio)
            all_intervals.append(before_intervals)
            all_intervals.append(during_intervals)
            all_intervals.append(after_intervals)
            event_occurrence[3*i] = event_occurrence[3*i+1] = event_occurrence[3*i+2] = len(hodf[(hodf['handoff_type.2'] == event_type) & (hodf['handoff_state'] != 'end')])
            # if anomaly_check:
                # anomaly_check_list.append(anomaly_check)
        print("disjoint set?", "yes!" if all_disjoint(all_intervals) else "no!")
        print("-----------------------------")

        all_intervals.append(all_union(all_intervals[:12:3]))
        all_intervals.append(all_union(all_intervals[1:12:3]))
        all_intervals.append(all_union(all_intervals[2:12:3]))
        all_intervals.append(all_union(all_intervals[:12]))
        all_intervals.append(P.closed(start_time, stop_time) - all_intervals[-1] if len(recvdf) else P.empty())
        all_intervals.append(P.closed(start_time, stop_time) if len(recvdf) else P.empty())

        loss_timestamps = lossdf['Timestamp'].array
        recv_timestamps = recvdf['Timestamp'].array
        for i, item in enumerate(all_intervals):
            sum_total_time[i] = get_intervals_length(item)
            # sum_total_num[i] = round(sum_total_time[i] * PKT_RATE)
            sum_total_num[i] = math.ceil(sum_total_time[i] * PKT_RATE)
            for j in range(len(lossdf)):
                if item.empty:
                    break
                # if lossdf['Timestamp'].iloc[j] > item.upper:
                if loss_timestamps[j] > item.upper:
                    break
                # if lossdf['Timestamp'].iloc[j] in item:
                if loss_timestamps[j] in item:
                    sum_loss_num[i] += 1
            ### 絕無誤差的計算 sent_packet 數量的方式（取消註解即可）
            # sum_total_num[i] = 0
            # for j in range(len(recvdf)):
            #     if item.empty:
            #         break
            #     # if recvdf['Timestamp'].iloc[j] > item.upper:
            #     if recv_timestamps[j] > item.upper:
            #         break
            #     # if recvdf['Timestamp'].iloc[j] in item:
            #     if recv_timestamps[j] in item:
            #         sum_total_num[i] += 1
            # sum_total_num[i] += sum_loss_num[i]
            ######
            loss_rate[i] = sum_loss_num[i] / (sum_total_num[i] + 1e-9) * 100
        
        event_occurrence[-4] = event_occurrence[-5] = event_occurrence[-6] = sum(event_occurrence[:12:3])  # successful handoff
        # event_occurrence[-4] = event_occurrence[-5] = sum(event_occurrence[24:30:2])  # failed handoff
        # event_occurrence[-3] = event_occurrence[-5] + event_occurrence[-8]  # all handoff (unstable)
        event_occurrence[-3] = event_occurrence[-6]  # all handoff (unstable)
        event_occurrence[-1] = event_occurrence[-2] = '-'  # stable & overall

        # datarow = [str(secs)]
        # for i in range(len(column_names)):
        #     datarow.append('@'.join([str(event_occurrence[i]), str(sum_total_time[i]), str(sum_loss_num[i]), str(sum_total_num[i]), str(loss_rate[i])]))
        datarow = ["{}-{}".format(secs0, secs1), "#event"] + event_occurrence
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "total_duration"] + sum_total_time
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "#packet_loss"] + sum_loss_num
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "#total_packets"] + sum_total_num
        datarows.append(datarow)
        datarow = ["{}-{}".format(secs0, secs1), "packet_loss_rate"] + loss_rate
        datarows.append(datarow)

    print("output >>>", fout)
    with open(fout, "w", newline='') as fp:
        writer = csv.writer(fp)
        # writer.writerow(["secs"] + column_names)
        writer.writerow(["secs", "metrics"] + column_names)
        writer.writerows(datarows)

# def loss_classify_type012(hodf, lossdf, recvdf, fout, start_time, stop_time, overlap=True, ratio=1):
#     # anomaly_check_list = []
#     handover_types_0 = "lte_handover,SN_addition,SN_removal,endc_SN_change,endc_MN_change,endc_MNSN_change,lte2endc_MN_change,endc2lte_MN_change".split(',')
#     handover_types_1 = "SN_change_only,Intra_frequency,Inter_frequency".split(',')
#     handover_types_2 = "SN_change_only,Intra_sector,Intra_eNB,Inter_eNB".split(',')
    
#     column_names = []
#     for type_name in handover_type_names:
#         column_names += ["before_{}".format(type_name), "during_{}".format(type_name), "after_{}".format(type_name)]
#     column_names += ["before_handoff", "during_handoff", "after_handoff", "unstable", "stable", "overall"]

#     datarows = []
#     ss = [i/10 for i in range(0, 10)] + list(range(1, 11))
#     for secs in ss:
#         event_occurrence = [0] * len(column_names)
#         sum_total_time = [0] * len(column_names)
#         sum_total_num = [0] * len(column_names)
#         sum_loss_num = [0] * len(column_names)
#         loss_rate = [0] * len(column_names)

#         all_intervals = []

#         offset = len(handover_type_names) * 3
#         for i, event_type in enumerate(handover_type_names):
#             before_intervals, during_intervals, after_intervals, anomaly_check = get_handover_intervals(hodf, types1=[event_type], secs=secs, overlap=overlap, ratio=ratio)
#             all_intervals.append(before_intervals)
#             all_intervals.append(during_intervals)
#             all_intervals.append(after_intervals)
#             event_occurrence[3*i] = event_occurrence[3*i+1] = event_occurrence[3*i+2] = len(hodf[(hodf['handoff_type'] == event_type) & (hodf['handoff_state'] != 'end')])
#             # if anomaly_check:
#                 # anomaly_check_list.append(anomaly_check)
#         print("disjoint set?", "yes!" if all_disjoint(all_intervals) else "no!")
#         print("-----------------------------")

#         all_intervals.append(all_union(all_intervals[:12:3]))
#         all_intervals.append(all_union(all_intervals[1:12:3]))
#         all_intervals.append(all_union(all_intervals[2:12:3]))
#         all_intervals.append(all_union(all_intervals[:12]))
#         all_intervals.append(P.closed(start_time, stop_time) - all_intervals[-1] if len(recvdf) else P.empty())
#         all_intervals.append(P.closed(start_time, stop_time) if len(recvdf) else P.empty())

#         loss_timestamps = lossdf['Timestamp'].array
#         for i, item in enumerate(all_intervals):
#             sum_total_time[i] = get_intervals_length(item)
#             sum_total_num[i] = round(sum_total_time[i] * PKT_RATE)
#             for j in range(len(lossdf)):
#                 if item.empty:
#                     break
#                 # if lossdf['Timestamp'].iloc[j] > item.upper:
#                 if loss_timestamps[j] > item.upper:
#                     break
#                 # if lossdf['Timestamp'].iloc[j] in item:
#                 if loss_timestamps[j] in item:
#                     sum_loss_num[i] += 1
#             ### 絕無誤差的計算 sent_packet 數量的方式（取消註解即可）
#             # sum_total_num[i] = 0
#             # for j in range(len(recvdf)):
#             #     if item.empty:
#             #         break
#             #     if recvdf['Timestamp'].iloc[j] > item.upper:
#             #         break
#             #     if recvdf['Timestamp'].iloc[j] in item:
#             #         sum_total_num[i] += 1
#             # sum_total_num[i] += sum_loss_num[i]
#             loss_rate[i] = sum_loss_num[i] / (sum_total_num[i] + 1e-9) * 100
        
#         event_occurrence[-4] = event_occurrence[-5] = event_occurrence[-6] = sum(event_occurrence[:12:3])  # successful handoff
#         # event_occurrence[-4] = event_occurrence[-5] = sum(event_occurrence[24:30:2])  # failed handoff
#         # event_occurrence[-3] = event_occurrence[-5] + event_occurrence[-8]  # all handoff (unstable)
#         event_occurrence[-3] = event_occurrence[-6]  # all handoff (unstable)
#         event_occurrence[-1] = event_occurrence[-2] = '-'  # stable & overall

#         datarow = [str(secs)]
#         for i in range(len(column_names)):
#             datarow.append('@'.join([str(event_occurrence[i]), str(sum_total_time[i]), str(sum_loss_num[i]), str(sum_total_num[i]), str(loss_rate[i])]))
#         datarows.append(datarow)

#     print("output >>>", fout)
#     with open(fout, "w", newline='') as fp:
#         writer = csv.writer(fp)
#         writer.writerow(["secs"] + column_names)
#         writer.writerows(datarows)

In [6]:
def main():
    # df_hndoff = pd.read_csv(os.path.join(source_dir, "diag_log_ho-info.csv"))
    df_hndoff = pd.read_csv(os.path.join(source_dir, "diag_log_ho-info_new.csv"))
    df_dlloss = pd.read_csv(os.path.join(source_dir, "udp_dnlk_loss_timestamp.csv"))
    df_dlrecv = pd.read_csv(os.path.join(source_dir, "udp_dnlk_latency.csv"))
    df_ulloss = pd.read_csv(os.path.join(source_dir, "udp_uplk_loss_timestamp.csv"))
    df_ulrecv = pd.read_csv(os.path.join(source_dir, "udp_uplk_latency.csv"))

    df_hndoff["Timestamp"] = df_hndoff["Timestamp"].apply(lambda x: pd.to_datetime(x))
    df_dlloss["Timestamp"] = df_dlloss["Timestamp"].apply(lambda x: pd.to_datetime(x))
    df_dlrecv["Timestamp"] = df_dlrecv["Timestamp"].apply(lambda x: pd.to_datetime(x))
    df_ulloss["Timestamp"] = df_ulloss["Timestamp"].apply(lambda x: pd.to_datetime(x))
    df_ulrecv["Timestamp"] = df_ulrecv["Timestamp"].apply(lambda x: pd.to_datetime(x))

    start, stop, expr_time = get_sst(df_dlloss, df_dlrecv, df_ulloss, df_ulrecv)
    # df_hndoff = remove_head_tail(df_hndoff, start, stop)
    # fout1 = os.path.join(target_dir, "diag_log_ho-statistics_new.csv")
    # fout2 = os.path.join(source_dir, "diag_log_ho-info_new.csv")
    # handoff_statistics(df_hndoff, expr_time, fout1, fout2)

    makedir(os.path.join(target_dir, "classify-uni"))

    fout = os.path.join(target_dir, "classify-uni", "uplk_loss_classify_uni_type0.csv")
    loss_classify_type0(df_hndoff.copy(), df_ulloss.copy(), df_ulrecv.copy(), fout, start, stop, overlap=False, ratio=0.5)
    fout = os.path.join(target_dir, "classify-uni", "dnlk_loss_classify_uni_type0.csv")
    loss_classify_type0(df_hndoff.copy(), df_dlloss.copy(), df_dlrecv.copy(), fout, start, stop, overlap=False, ratio=0.5)

    fout = os.path.join(target_dir, "classify-uni", "uplk_loss_classify_uni_type1.csv")
    loss_classify_type1(df_hndoff.copy(), df_ulloss.copy(), df_ulrecv.copy(), fout, start, stop, overlap=False, ratio=0.5)
    fout = os.path.join(target_dir, "classify-uni", "dnlk_loss_classify_uni_type1.csv")
    loss_classify_type1(df_hndoff.copy(), df_dlloss.copy(), df_dlrecv.copy(), fout, start, stop, overlap=False, ratio=0.5)

    fout = os.path.join(target_dir, "classify-uni", "uplk_loss_classify_uni_type2.csv")
    loss_classify_type2(df_hndoff.copy(), df_ulloss.copy(), df_ulrecv.copy(), fout, start, stop, overlap=False, ratio=0.5)
    fout = os.path.join(target_dir, "classify-uni", "dnlk_loss_classify_uni_type2.csv")
    loss_classify_type2(df_hndoff.copy(), df_dlloss.copy(), df_dlrecv.copy(), fout, start, stop, overlap=False, ratio=0.5)


# ******************************* Check Files *********************************
for expr, (times, traces) in exps.items():
    print(os.path.join(database, date, expr))
    for dev in devices:
        if not os.path.isdir(os.path.join(database, date, expr, dev)):
            print("|___ {} does not exist.".format(os.path.join(database, date, expr, dev)))
            continue
        
        print("|___", os.path.join(database, date, expr, dev))
        if traces == None:
            # print(os.path.join(database, date, expr, dev))
            continue
        elif len(traces) == 0:
            traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
        
        print("|    ", times)
        traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
        if len(traces) != times:
            print("***************************************************************************************")
            print("Warning: the number of traces does not match the specified number of experiment times.")
            print("***************************************************************************************")
        for trace in traces:
            print("|    |___", os.path.join(database, date, expr, dev, trace))
    print()
# *****************************************************************************

# ******************************** Processing *********************************
t = TicToc()  # create instance of class
t.tic()       # Start timer
err_handles = []
for expr, (times, traces) in exps.items():
    # print("Haaaaaaaaa")
    for dev in devices:
        # print("Ha000000000")
        if not os.path.isdir(os.path.join(database, date, expr, dev)):
            print("{} does not exist.\n".format(os.path.join(database, date, expr, dev)))
            continue
        # print("Ha0011111111")

        if traces == None:
            print("------------------------------------------")
            print(date, expr, dev)
            print("------------------------------------------")
            source_dir = os.path.join(database, date, expr, dev)
            target_dir = os.path.join(database, date, expr, dev)
            makedir(target_dir)
            filenames = os.listdir(source_dir)
            main()
            continue
        elif len(traces) == 0:
            traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
        
        traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
        # print("Ha00222222")
        print(traces)
        for trace in traces:
            # print("Ha033333333")
            print("------------------------------------------")
            print(date, expr, dev, trace)
            print("------------------------------------------")
            source_dir = os.path.join(database, date, expr, dev, trace, "data")
            target_dir = os.path.join(database, date, expr, dev, trace, "statistics")
            makedir(target_dir)
            main()
t.toc()  # Time elapsed since t.tic()
# *****************************************************************************

/home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3
|___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm05
|     4
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm05/#01
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm05/#02
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm05/#03
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm05/#04
|___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm06
|     4
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm06/#01
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm06/#02
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm06/#03
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm06/#04
|___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm07
|     4
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B1_B3/sm07/#01
|    |___ /home/wmnlab/D/database/2022-11-29/_Bandlock_Udp_B